# GDAS

## 0. Paper

### Info
* Title: Searching for A Robust Neural Architecture in Four GPU Hours
* Author: Xuanyi Dong
* Task: Neural Architecture Search
* Link: https://arxiv.org/abs/1910.04465


### Features
* Dataset: CIFAR-10

### Reference
* https://github.com/D-X-Y/AutoDL-Projects

## 1. Setting

In [1]:
import os
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchsummary import torchsummary

In [2]:
class CONFIG:
    batch_size = 128
    epoch_size = 100
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dataset = 'CIFAR-10'
    base_dir = '/content/drive/Shared drives/Yoon/Project/Doing/Deep Learning Paper Implementation'

## 2. Data

In [7]:
def create_dataloader():
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2439, 0.2616))
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2439, 0.2616))
    ])


    indices = np.random.permutation(50000)
    train_indices, valid_indices = indices[:25000], indices[25000:]
    train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
    valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(valid_indices)

    train_data = torchvision.datasets.CIFAR10('./data', train=True, transform=train_transform, download=True)
    valid_data = torchvision.datasets.CIFAR10('./data', train=True, transform=test_transform, download=True)
    test_data = torchvision.datasets.CIFAR10('./data', train=False, transform=test_transform, download=True)

    train_loader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=CONFIG.batch_size)
    valid_loader = torch.utils.data.DataLoader(valid_data, sampler=valid_sampler, batch_size=CONFIG.batch_size)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=CONFIG.batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader

In [8]:
train_loader, valid_loader, test_loader = create_dataloader()
x, y = next(iter(train_loader))
print(x.size(), y.size())

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
torch.Size([128, 3, 32, 32]) torch.Size([128])


## 3. Model

In [168]:
class Identity(nn.Module):
    def forward(self, x):
        return x

class Zero(nn.Module):
    def __init__(self, C_in, C_out, stride):
        super(Zero, self).__init__()
        self.C_in = C_in
        self.C_out = C_out
        self.stride = stride
    
    def forward(self, x):
        if self.C_in == self.C_out:
            x = x[:, :, ::self.stride, ::self.stride]
            return x.mul(0.)
        else:
            shape = list(x.size())
            shape[1] = self.C_out
            shape[2] //= self.stride
            shape[3] //= self.stride
            zeros = x.new_zeros(shape, dtype=x.dtype, device=x.device)
            return zeros


class ReLUConvBN(nn.Module):
    def __init__(self, C_in, C_out, kernel_size, stride, padding, dilation, affine, track_running_stats):
        super(ReLUConvBN, self).__init__()
        self.op = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(C_in, C_out, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, bias=not affine),
            nn.BatchNorm2d(C_out, affine=affine, track_running_stats=track_running_stats)
        )
    
    def forward(self, x):
        return self.op(x)


class Pooling(nn.Module):
    def __init__(self, C_in, C_out, stride, mode, affine=True, track_running_stats=True):
        super(Pooling, self).__init__()
        if C_in == C_out:
            self.prep = None
        else:
            self.prep = ReLUConvBN(C_in, C_out, 1, 1, 0, 1, affine, track_running_stats)
        
        if mode == 'avg':
            self.op = nn.AvgPool2d(3, stride=stride, padding=1, count_include_pad=False)
        elif mode == 'max':
            self.op = nn.MaxPool2d(3, stride=stride, padding=1)
        else:
            raise ValueError(f'Invalid mode: {mode}')
    
    def forward(self, x):
        if self.prep:
            x = self.prep(x)
        return self.op(x)


class FactorizedReduce(nn.Module):
    def __init__(self, C_in, C_out, stride, affine, track_running_stats):
        super(FactorizedReduce, self).__init__()
        self.stride = stride

        if stride == 2:
            self.relu = nn.ReLU()
            self.pad = nn.ConstantPad2d((0, 1, 0, 1), 0)
            C_outs = [C_out // 2, C_out - C_out // 2]
            self.conv = nn.ModuleList([nn.Conv2d(C_in, C_outs[i], kernel_size=1, stride=stride, padding=0, bias=not affine) for i in range(2)])
        elif stride == 1:
            self.conv = nn.Conv2d(C_in, C_out, kernel_size=1, stride=stride, padding=0, bias=not affine)

        self.bn = nn.BatchNorm2d(C_out, affine=affine, track_running_stats=track_running_stats)
    
    def forward(self, x):
        if self.stride == 2:
            x1 = self.relu(x)
            x2 = self.pad(x1)
            out = torch.cat([self.conv[0](x1), self.conv[1](x2[:,:,1:,1:])], dim=1)
        else:
            out = self.conv(x)
        
        out = self.bn(out)
        return out


class ResNetBasicBlock(nn.Module):
    def __init__(self, in_C, C, stride, affine=True, track_running_stats=True):
        super(ResNetBasicBlock, self).__init__()
        self.conv1 = ReLUConvBN(in_C, C, kernel_size=3, stride=stride, padding=1, dilation=1, affine=affine, track_running_stats=track_running_stats)
        self.conv2 = ReLUConvBN(C, C, kernel_size=3, stride=1, padding=1, dilation=1, affine=affine, track_running_stats=track_running_stats)

        self.shortcut = nn.Sequential()
        if stride == 2 or in_C != C:
            self.shortcut = ReLUConvBN(in_C, C, kernel_size=1, stride=stride, padding=0, dilation=1, affine=affine, track_running_stats=track_running_stats)
    
    def forward(self, x):
        shortcut = self.shortcut(x)
        x = self.conv1(x)
        x = self.conv2(x)
        return shortcut + x

class ReductionCell(nn.Module):
    def __init__(self, C_prev_prev, C_prev, C, reduction_prev, affine, track_running_stats):
        super(ReductionCell, self).__init__()
        self.multiplier = 4

        if reduction_prev:
            self.prep0 = FactorizedReduce(C_prev_prev, C, 2, affine, track_running_stats)
        else:
            self.prep0 = ReLUConvBN(C_prev_prev, C, 1, 1, 0, 1, affine, track_running_stats)
        self.prep1 = ReLUConvBN(C_prev, C, 1, 1, 0, 1, affine, track_running_stats)

        self.ops1 = nn.ModuleList([
            nn.Sequential(
                nn.ReLU(),
                nn.Conv2d(C, C, kernel_size=(1, 3), stride=(1, 2), padding=(0, 1), groups=8, bias=not affine),
                nn.Conv2d(C, C, kernel_size=(3, 1), stride=(2, 1), padding=(1, 0), groups=8, bias=not affine),
                nn.BatchNorm2d(C, affine=affine, track_running_stats=track_running_stats),
                nn.ReLU(),
                nn.Conv2d(C, C, kernel_size=1, padding=0, bias=not affine),
                nn.BatchNorm2d(C, affine=affine, track_running_stats=track_running_stats)
            ),

            nn.Sequential(
                nn.ReLU(),
                nn.Conv2d(C, C, kernel_size=(1, 3), stride=(1, 2), padding=(0, 1), groups=8, bias=not affine),
                nn.Conv2d(C, C, kernel_size=(3, 1), stride=(2, 1), padding=(1, 0), groups=8, bias=not affine),
                nn.BatchNorm2d(C, affine=affine, track_running_stats=track_running_stats),
                nn.ReLU(),
                nn.Conv2d(C, C, kernel_size=1, padding=0, bias=not affine),
                nn.BatchNorm2d(C, affine=affine, track_running_stats=track_running_stats)
            ),
        ])

        self.ops2 = nn.ModuleList([
            nn.Sequential(
                nn.MaxPool2d(3, stride=2, padding=1),
                nn.BatchNorm2d(C, affine=affine, track_running_stats=track_running_stats)
            ),

            nn.Sequential(
                nn.MaxPool2d(3, stride=2, padding=1),
                nn.BatchNorm2d(C, affine=affine, track_running_stats=track_running_stats)
            )
        ])
    
    def forward(self, s0, s1, drop_prob=-1):
        s0 = self.prep0(s0)
        s1 = self.prep1(s1)

        X0 = self.ops1[0](s0)
        X1 = self.ops1[1](s1)
        
        X2 = self.ops2[0](s0)
        X3 = self.ops2[1](s1)
        if self.training and drop_prob > 0:
            X2, X3 = drop_path(X2, drop_prob), drop_path(X3, drop_prob)

        return torch.cat([X0, X1, X2, X3], dim=1)


class SmallSearchCell(nn.Module):
    def __init__(self, C_in, C_out, stride, max_nodes, op_names, affine=False, track_running_stats=True):
        super(SearchCell, self).__init__()
        self.max_nodes = max_nodes


        self.edges = nn.ModuleDict()
        for i in range(1, max_nodes):
            for j in range(i):
                node_str = f'{i}<-{j}'
                _stride = stride if j ==0 else 1
                xlists = [OPS[op_name](C_in, C_out, _stride, affine, track_running_stats) for op_name in op_names]
                self.edges[node_str] = nn.ModuleList(xlists)
        
        self.edge_keys = sorted(list(self.edges.keys()))
        self.edge2index = {key:i for i, key in enumerate(self.edge_keys)}
        self.num_edges = len(self.edges)

    
    def forward(self, inputs, indices):
        nodes = [inputs]
        for i in range(1, self.max_nodes):
            inter_nodes = []
            for j in range(i):
                node_str = f'{i}<-{j}'
                #weights = hard_weights[self.edge2index[node_str]]
                idx = indices[self.edge2index[node_str]].item()
                #node = sum(weights[idx] * edge(nodes[j]) if _idx == argmaxs else weights[idx] for idx, edge in enumerate(self.edges[node_str]))
                node = self.edges[node_str][idx](nodes[j])
                inter_nodes.append(node)
            nodes.append(sum(inter_nodes))
        return nodes[-1]

In [169]:
class MixedOp(nn.Module):
    def __init__(self, space, C, stride, affine, track_running_stats):
        super(MixedOp, self).__init__()
        self.ops = nn.ModuleList()
        for primitive in space:
            op = OPS[primitive](C, C, stride, affine, track_running_stats)
            self.ops.append(op)
    
    def forward(self, inputs, weight, index):
        return self.ops[index](inputs) * weight[index]

    # DARTS forward
    # def forward(self, x, weights):
    #     return sum(w * op(x), w, op in zip(weights, self.ops))

In [170]:
class SearchCell(nn.Module):
    def __init__(self, space, steps, multiplier, C_prev_prev, C_prev, C, reduction, reduction_prev, affine, track_running_stats):
        super(SearchCell ,self).__init__()
        self.steps = steps
        self.multiplier = multiplier

        if reduction_prev:
            self.prep0 = FactorizedReduce(C_prev_prev, C, 2, affine, track_running_stats)
        else:
            self.prep0 = ReLUConvBN(C_prev_prev, C, 1, 1, 0, 1, affine, track_running_stats)
        self.prep1 = ReLUConvBN(C_prev, C, 1, 1, 0, 1, affine, track_running_stats)

        self.edges = nn.ModuleDict()
        for i in range(steps):
            for j in range(2+i):
                node_str = f'{i}<-{j}'
                stride = 2 if reduction and j < 2 else 1
                op = MixedOp(space, C, stride, affine, track_running_stats)
                self.edges[node_str] = op
        
        self.edge_keys = sorted(list(self.edges.keys()))
        self.edge2index = {key: i for i, key in enumerate(self.edge_keys)}
        self.num_edges = len(self.edges)


    def forward(self, s0, s1, weights, indices):
        s0 = self.prep0(s0)
        s1 = self.prep1(s1)
        
        states = [s0, s1]
        for i in range(self.steps):
            clist = []
            for j, h in enumerate(states):
                node_str = f'{i}<-{j}'
                op = self.edges[node_str]
                weight = weights[self.edge2index[node_str]]
                index = indices[self.edge2index[node_str]].item()
                clist.append(op(h, weight, index))
            states.append(sum(clist))
        
        return torch.cat(states[-self.multiplier:], dim=1)

In [171]:
OPS = {
    'none': lambda C_in, C_out, stride, affine, track_running_stats: Zero(C_in, C_out, stride),
    'skip_connect': lambda C_in, C_out, stride, affine, track_running_stats: Identity() if stride == 1 and C_in == C_out else FactorizedReduce(C_in, C_out, stride, affine, track_running_stats),
    'avg_pool_3x3': lambda C_in, C_out, stride, affine, track_running_stats: Pooling(C_in, C_out, stride, 'avg', affine, track_running_stats),
    'max_pool_3x3': lambda C_in, C_out, stride, affine, track_running_stats: Pooling(C_in, C_out, stride, 'max', affine, track_running_stats),
    'conv_7x7': lambda C_in, C_out, stride, affine, track_running_stats: ReLUConvBN(C_in, C_out, 7, stride, 3, 1, affine, track_running_stats),
    'conv_3x3': lambda C_in, C_out, stride, affine, track_running_stats: ReLUConvBN(C_in, C_out, 3, stride, 1, 1, affine, track_running_stats),
    'conv_1x1': lambda C_in, C_out, stride, affine, track_running_stats: ReLUConvBN(C_in, C_out, 1, stride, 0, 1, affine, track_running_stats),
}

In [195]:
# 왜  reduction cell 안쓰지

class GDAS(nn.Module):
    def __init__(self, C, N, max_nodes, num_classes, search_space, affine, track_running_stats):
        super(GDAS, self).__init__()
        self.steps = 4
        self.multiplier = 4

        self.stem = nn.Sequential(
            nn.Conv2d(3, C, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(C)
        )

        layer_channels = [C] * N + [C*2] * (N+1) + [C*4] * (N+1)
        layer_reductions = [False] * N + [True] + [False] * N + [True] + [False] * N
        layer_reductions_prev = [False] * (N+1) + [True] + [False] * (N) + [True] + [False] * (N-1)
        C_prev_prev, C_prev, num_edge, edge2index = C, C, None, None
        
        self.cells = nn.ModuleList()
        for index, (C_curr, reduction, reduction_prev) in enumerate(zip(layer_channels, layer_reductions, layer_reductions_prev)):
            if reduction:
                cell = ReductionCell(C_prev_prev, C_prev, C_curr, reduction_prev, affine, track_running_stats)
            else:
                #cell = SearchCell(C_prev, C_curr, 1, max_nodes, search_space, affine, track_running_stats)
                cell = SearchCell(search_space, self.steps, self.multiplier, C_prev_prev, C_prev, C_curr, reduction, reduction_prev, affine, track_running_stats)
            
            if num_edge is None:
                num_edge, edge2index = cell.num_edges, cell.edge2index
            
            self.cells.append(cell)
            C_prev_prev, C_prev = C_prev, self.multiplier * C_curr

        self.classifier = nn.Sequential(
            nn.BatchNorm2d(C_prev),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(C_prev, num_classes)
        )

        self.arch_parameters = nn.Parameter(1e-3 * torch.randn(num_edge, len(search_space)))

    def set_tau(self, tau):
        self.tau = tau

    def get_weights(self):
        return list(self.stem.parameters()) + list(self.cells.parameters()) + list(self.classifier.parameters())

    def get_alphas(self):
        return [self.arch_parameters]


    def forward(self, inputs):
        weights = F.gumbel_softmax(self.arch_parameters, self.tau, hard=False)
        indices = weights.max(1, keepdim=True)[1]

        s0 = s1 = self.stem(inputs)
        for i, cell in enumerate(self.cells):
            if isinstance(cell, SearchCell):
                s0, s1 = s1, cell(s0, s1, weights, indices)
            else:
                s0, s1 = s1, cell(s0, s1)
        logits = self.classifier(s1)
        return logits

## 4. Experiment

In [184]:
class AverageMeter(object):
    def __init__(self, name):
        self.name = name
        self.reset()

    def reset(self):
        self.sum = 0
        self.count = 0
        self.avg = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = f'{self.name:10s} {self.avg:.3f}'
        return fmtstr


class ProgressMeter(object):
    def __init__(self, meters):
        self.meters = [AverageMeter(m) for m in meters]
    
    def reset(self):
        for m in self.meters:
            m.reset()
    
    def update(self, values, n=1):
        for m, v in zip(self.meters, values):
            m.update(v, n)
            self.__setattr__(m.name, m.avg)

    def log(self):
        msg = [str(meter) for meter in self.meters]
        msg = ' | '.join(msg)
        return msg


def accuracy(logits, targets, k=[1]):
    maxk = max(k)
    _, preds = logits.topk(maxk, dim=1) # (batch_size, maxk)
    targets = targets.unsqueeze(1).expand_as(preds) # (batch_size, maxk)
    corrects = preds.eq(targets)

    result = []
    for _k in k:
        res = corrects[:,:_k].any(dim=1).float().mean().item()
        result.append(res)
        
    return result

In [185]:
class SearchTrainer(object):
    def __init__(self, model, criterion, w_optimizer, w_scheduler, a_optimizer, tau_max, tau_min):
        self.model = model.to(CONFIG.device)
        self.criterion = criterion.to(CONFIG.device)
        self.w_optimizer = w_optimizer
        self.w_scheduler = w_scheduler
        self.a_optimizer = a_optimizer
        self.tau_max = tau_max
        self.tau_min = tau_min

    def train(self, train_loader, valid_loader, epoch):
        progress = ProgressMeter(["train_loss", "train_top1", "train_top5", "valid_loss", "valid_top1", "valid_top5"])
        self.model.train()

        tau = self.tau_max - (self.tau_max - self.tau_min) * epoch / (CONFIG.epoch_size - 1)
        self.model.set_tau(tau)

        pbar = tqdm(zip(train_loader, valid_loader), total=len(train_loader))
        pbar.set_description(f'TRAIN {epoch:03d}')
        for idx, ((train_inputs, train_targets), (valid_inputs, valid_targets)) in enumerate(pbar):
            train_inputs, train_targets = train_inputs.to(CONFIG.device), train_targets.to(CONFIG.device)
            valid_inputs, valid_targets = valid_inputs.to(CONFIG.device), valid_targets.to(CONFIG.device)
    
            train_logits = self.model(train_inputs)
            train_loss = self.criterion(train_logits, train_targets)
            self.w_optimizer.zero_grad()
            train_loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 5)
            self.w_optimizer.step()

            valid_logits = self.model(valid_inputs)
            valid_loss = self.criterion(valid_logits, valid_targets)
            self.a_optimizer.zero_grad()
            valid_loss.backward()
            self.a_optimizer.step()

            train_loss = train_loss.item()
            train_top1, train_top5 = accuracy(train_logits, train_targets, k=[1, 5])
            valid_loss = valid_loss.item()
            valid_top1, valid_top5 = accuracy(valid_logits, valid_targets, k=[1, 5])

            progress.update([train_loss, train_top1, train_top5, valid_loss, valid_top1, valid_top5], n=train_inputs.size(0))
            pbar.set_postfix(log=progress.log())

        self.w_scheduler.step()

In [186]:
channels = 16
num_cells = 5
max_nodes = 4
tau_max = 10
tau_min = 0.1

num_classes = 10
affine = False
track_running_stats = True
search_space = ['none', 'skip_connect', 'avg_pool_3x3', 'max_pool_3x3', 'conv_7x7', 'conv_3x3', 'conv_1x1']

In [196]:
model = GDAS(channels, num_cells, max_nodes, num_classes, search_space, affine, track_running_stats)
criterion = nn.CrossEntropyLoss()
w_optimizer = torch.optim.SGD(model.get_weights(), lr=0.025, momentum=0.9, weight_decay=5e-4, nesterov=True) # weight optimizer
w_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(w_optimizer, CONFIG.epoch_size, eta_min=1e-3)
a_optimizer = torch.optim.Adam(model.get_alphas(), lr=3e-4, betas=(0.5, 0.999), weight_decay=1e-3) # architecture optimizer

In [198]:
trainer = SearchTrainer(model, criterion, w_optimizer, w_scheduler, a_optimizer, tau_max, tau_min)

In [199]:
for ep in range(CONFIG.epoch_size):
    trainer.train(train_loader, valid_loader, ep)